In [13]:
import math
import random
import numpy as np
from datetime import datetime
import os
# from firebase_admin import credentials, firestore, initialize_app
import string 
from sklearn.cluster import KMeans
from nsetools import Nse
nse = Nse()

#risk option weights for each option
risk_option_weights=[[0,2,5,8],[1,3,7,10],[0,1,4,8],[3,6,8,0],[0,4,8,0],[1,3,7,8]]

#initialize firebase
# notebook_path = os.path.abspath("cluster.ipynb")
# cred = credentials.Certificate(os.path.join(os.path.dirname(notebook_path), "fpma.json"))
# default_app = initialize_app(cred)
# db = firestore.client()

#get all NSE stock symbols
all_stock_codes = nse.get_stock_codes()
stock_symbols=[];
for s in all_stock_codes:
    stock_symbols.append(s)
stock_symbols.pop(0)
stock_symbols = random.choices (stock_symbols,k=50)

# stock_symbols = [""]

In [14]:
#function to create dummy users
def make_users(size):
    users=[]
    
    #adding size number of users in users array
    for x in range(size):
        user={}
        
        #get random watch_list_symbols of size 10 for each user from stock symbols extracted from NSE
        watch_list_symbols = random.choices(stock_symbols,k=10)
        
        #random use_id
        user_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 20))
        
        riskScore = []
        #generate random riskData for size 6
        risk_data = np.random.randint(4, size=6)
        
        #get riskScore for each option number selected by random user
        for i,r in enumerate(risk_data):
            riskScore.append(risk_option_weights[i][r])
            
        #create user and append to users array
        user["user_id"] = user_id
        user["riskScore"]=riskScore
        user["watch_list_symbols"]=watch_list_symbols
        users.append(user)
        
    return users


# users-users data
# k-number of clusters we want
def make_clusters(users,k):
    
    
    #all clusters will be stores in clusters
    clusters={}
    
    #add k-blank clusters 
    for i in range(k):
        clusters["cluster_"+str(i)]=[]
        
    #extract riskScores from users data
    riskScores=[]
    for user in users:
        riskScores.append(user["riskScore"])
    
    #get cluster labels based on riskScores
    kmeans = KMeans(n_clusters=5, random_state=0).fit(riskScores)
    cluster_labels = kmeans.labels_

    for x in range(len(users)):
        #add label to corresponding user
        users[x]["cluster_label"] = cluster_labels[x]
        #add user to its cluster based on 
        clusters["cluster_"+str(cluster_labels[x])].append(users[x]);
        
    #return users and clusters
    return users,clusters
 

    




#Euclidian distance between two riskScores
def euclidean_distance(riskScore1, riskScore2):
    sum_squared_distance = 0
    for i in range(len(riskScore1)):
        sum_squared_distance += math.pow(riskScore1[i] - riskScore2[i], 2)
    return math.sqrt(sum_squared_distance)


#making NN matrix for each cluster separatly
#k-number of NN we want
def make_NN_map(cluster,k):
    user_user_distances=[]
    
    #map of user_id and nearest neighbours
    NN_map={}
    
    for i in range(len(cluster)):
        user_i_distances=[]
        user_i = cluster[i]
        for j in range(len(cluster)):
            user_j = cluster[j]
            
            #calculate euclidian distancee between riskScores of (user_i) and (user_j)
            distance_ij = euclidean_distance(user_i["riskScore"], user_j["riskScore"])
            
            #add distance_ij distance to user_i's distance array with index of user_j
            user_i_distances.append((distance_ij,j))
            
        #append user_i's distances to user_user_distance matrix
        user_user_distances.append(user_i_distances)
            
            
    for i in range(len(user_user_distances)):
        #sort the distance array of user_i in accesding order ie:less distace first, we have preserved the index while sorting
        sorted_user_i_distances = sorted(user_user_distances[i])
        
        #select first k distances from sorted user_i's distance array
        k_sorted_user_i_distances = sorted_user_i_distances[:k]
        
        #add these k sorted distances to user_user_distances matrix
        user_user_distances[i] = k_sorted_user_i_distances
        
        #finally get the users from cluster using the index which we preserved
        #and add them as array of users to NN map 
        knn_to_user_i = [cluster[j] for distance, j in k_sorted_user_i_distances]
        NN_map[cluster[i]["user_id"]]=knn_to_user_i;
        
    
    return user_user_distances,NN_map
    
    
        

In [15]:
users = make_users(3000)
for u in users:
    print(u)
    print()

{'user_id': '2235MPB9YSB46TPBJREF', 'riskScore': [0, 3, 4, 3, 0, 7], 'watch_list_symbols': ['MANUGRAPH', 'AXISCADES', 'HDIL', 'INDOTECH', 'MEGH', 'CONCOR', 'MUTHOOTCAP', 'RELIGARE', 'BCG', 'ADHUNIKIND']}

{'user_id': 'L6WQIPKA0LS10AEHDP2S', 'riskScore': [8, 1, 8, 6, 0, 7], 'watch_list_symbols': ['EMAMIREAL', 'MANUGRAPH', 'AXISCADES', 'LYKALABS', 'JHS', 'AARTISURF', 'GOACARBON', 'LOTUSEYE', 'PUNJLLOYD', 'TVSSRICHAK']}

{'user_id': 'JRX606HT2R4HMFEJ6YEC', 'riskScore': [5, 7, 4, 6, 4, 8], 'watch_list_symbols': ['GOACARBON', 'MAHABANK', 'SUPERHOUSE', 'GAL', 'MANUGRAPH', 'AARTISURF', 'INDOTECH', 'MAHABANK', 'JHS', 'JHS']}

{'user_id': 'OLIQPLKI2NGIR665LA6N', 'riskScore': [2, 10, 4, 3, 4, 1], 'watch_list_symbols': ['ASHIMASYN', 'PUNJLLOYD', 'AYMSYNTEX', 'JAGRAN', 'DIGISPICE', 'LOTUSEYE', 'TATASTLBSL', 'LOTUSEYE', 'HUBTOWN', 'ESABINDIA']}

{'user_id': 'KIDRRJID8UG1KYW0WZ27', 'riskScore': [8, 7, 0, 3, 4, 1], 'watch_list_symbols': ['TATASTLBSL', 'TVSSRICHAK', 'RUPA', 'SMARTLINK', 'ENDURANCE', '


{'user_id': '1RC5ZIQSR6JFD5YQKZVW', 'riskScore': [8, 7, 0, 3, 0, 3], 'watch_list_symbols': ['BCG', 'VIDHIING', 'BALAXI', 'DHUNINV', 'RELIGARE', 'EMAMIREAL', 'GOACARBON', 'MUTHOOTCAP', 'TATASTLBSL', 'INDOTECH']}

{'user_id': 'UTVMAFQUQ29EA42EQ6A4', 'riskScore': [5, 7, 0, 3, 4, 8], 'watch_list_symbols': ['AUTOAXLES', 'BALAXI', 'JHS', 'DIGISPICE', 'LYKALABS', 'SMARTLINK', 'CAPTRUST', 'ARCHIDPLY', 'INDOTECH', 'VIDHIING']}

{'user_id': 'UD9E7IFVNO4B3RRRN026', 'riskScore': [2, 10, 4, 8, 8, 3], 'watch_list_symbols': ['ARCHIDPLY', 'SRPL', 'ESSARSHPNG', 'LYKALABS', 'INDOSOLAR', 'MANUGRAPH', 'LOTUSEYE', 'HDIL', 'SMARTLINK', 'RPOWER']}

{'user_id': '9M5O7V5ZR6SH6986CUWE', 'riskScore': [2, 10, 8, 3, 0, 3], 'watch_list_symbols': ['INDOTECH', 'EMAMIREAL', 'MEGH', 'INDOTECH', 'LYKALABS', 'LYKALABS', 'RUPA', 'SMARTLINK', 'MANAKALUCO', 'AARTISURF']}

{'user_id': 'S8S0XZ9BWTJTTCCENG6D', 'riskScore': [8, 1, 4, 0, 0, 3], 'watch_list_symbols': ['HDIL', 'CONCOR', 'GOLDENTOBC', 'ENDURANCE', 'VIDHIING', 'BAL

In [16]:
users,clusters = make_clusters(users,5)

for cluster_name,cluster in clusters.items():
    print(cluster_name)
    print(len(cluster))
    for user in cluster:
        print(user)
        

cluster_0
729
{'user_id': '2235MPB9YSB46TPBJREF', 'riskScore': [0, 3, 4, 3, 0, 7], 'watch_list_symbols': ['MANUGRAPH', 'AXISCADES', 'HDIL', 'INDOTECH', 'MEGH', 'CONCOR', 'MUTHOOTCAP', 'RELIGARE', 'BCG', 'ADHUNIKIND'], 'cluster_label': 0}
{'user_id': 'T9IUF16D3ENK114JEFEI', 'riskScore': [5, 3, 1, 6, 0, 8], 'watch_list_symbols': ['EMAMIREAL', 'CONCOR', 'SUPERHOUSE', 'ASHIMASYN', 'MANAKALUCO', 'SJVN', 'ARCHIDPLY', 'GEPIL', 'PUNJLLOYD', 'GOLDENTOBC'], 'cluster_label': 0}
{'user_id': 'DFDN8GEKAJCYJSIFUZML', 'riskScore': [8, 1, 1, 0, 0, 8], 'watch_list_symbols': ['MADRASFERT', 'BCG', 'INDOSOLAR', 'BCG', 'BALAXI', 'LYKALABS', 'INDOTECH', 'MANUGRAPH', 'GAL', 'SJVN'], 'cluster_label': 0}
{'user_id': '4AESJRLJG81Y9QVFTK2E', 'riskScore': [2, 1, 1, 6, 0, 8], 'watch_list_symbols': ['EROSMEDIA', 'DHUNINV', 'GOLDENTOBC', 'SRPL', 'GOACARBON', 'ESSARSHPNG', 'MAHABANK', 'BCG', 'MEGH', 'RPOWER'], 'cluster_label': 0}
{'user_id': 'H8KAPDX43XPS3KSBG7NH', 'riskScore': [2, 1, 1, 8, 4, 1], 'watch_list_symbols'

{'user_id': 'CV6N75YH28ZW9N863C8L', 'riskScore': [0, 10, 0, 0, 0, 8], 'watch_list_symbols': ['AARTISURF', 'EROSMEDIA', 'AUTOAXLES', 'CAPTRUST', 'HDIL', 'GOACARBON', 'PTL', 'CONCOR', 'GOACARBON', 'MADRASFERT'], 'cluster_label': 4}
{'user_id': 'ZHFCZGP0G0LPAXJVCCHC', 'riskScore': [8, 7, 1, 8, 0, 7], 'watch_list_symbols': ['ASHIMASYN', 'HDIL', 'MADRASFERT', 'DIGISPICE', 'SJVN', 'GEPIL', 'PTL', 'HUBTOWN', 'AYMSYNTEX', 'GEPIL'], 'cluster_label': 4}
{'user_id': '8GJGQBYMQPUJ5L9NMBUG', 'riskScore': [2, 7, 4, 6, 0, 1], 'watch_list_symbols': ['AARTISURF', 'CONCOR', 'ARCHIDPLY', 'GOACARBON', 'CONCOR', 'AARTISURF', 'GEPIL', 'PUNJLLOYD', 'JAGRAN', 'SUPERHOUSE'], 'cluster_label': 4}
{'user_id': '9D7M0ZT61FX1EIOAFR7Q', 'riskScore': [2, 7, 1, 3, 4, 7], 'watch_list_symbols': ['AARTISURF', 'SJVN', 'AXISCADES', 'VIDHIING', 'SUPPETRO', 'LYKALABS', 'AUTOAXLES', 'VIDHIING', 'BCG', 'GOACARBON'], 'cluster_label': 4}
{'user_id': 'A8YPF815MUJ92D8E9GF1', 'riskScore': [8, 10, 4, 0, 0, 3], 'watch_list_symbols': [

In [17]:
_,NN_map = make_NN_map(clusters["cluster_1"],6)

In [18]:
for user_id,nn_users in NN_map.items():
    print(user_id+" ->")
    for user in nn_users:
        print(user)
        print()
    print("-------------------------------------------------------------------------------------------")

KRX8IB646X4LEIA19IU6 ->
{'user_id': 'KRX8IB646X4LEIA19IU6', 'riskScore': [0, 7, 4, 6, 8, 7], 'watch_list_symbols': ['ESABINDIA', 'INDOTECH', 'INDOSOLAR', 'AARTISURF', 'MANUGRAPH', 'RUPA', 'SUPERHOUSE', 'SUPERHOUSE', 'HDIL', 'EROSMEDIA'], 'cluster_label': 1}

{'user_id': '8UVR3T57PA53YBGXZRFW', 'riskScore': [0, 7, 4, 6, 8, 8], 'watch_list_symbols': ['MANUGRAPH', 'INDOSOLAR', 'MUTHOOTCAP', 'LYKALABS', 'ESSARSHPNG', 'DHUNINV', 'EMAMIREAL', 'EROSMEDIA', 'PUNJLLOYD', 'ESSARSHPNG'], 'cluster_label': 1}

{'user_id': 'TP03ZRJDUN0Z4OAHWEMW', 'riskScore': [0, 7, 4, 8, 8, 7], 'watch_list_symbols': ['SJVN', 'AYMSYNTEX', 'RELIGARE', 'CONCOR', 'MADRASFERT', 'LOTUSEYE', 'BALAXI', 'SJVN', 'ENDURANCE', 'SRPL'], 'cluster_label': 1}

{'user_id': 'KGZKNPPZUD4H8USR2649', 'riskScore': [2, 7, 4, 6, 8, 7], 'watch_list_symbols': ['BALAXI', 'HDIL', 'PTL', 'MADRASFERT', 'RUPA', 'ENDURANCE', 'HUBTOWN', 'HUBTOWN', 'GOACARBON', 'SRPL'], 'cluster_label': 1}

{'user_id': 'HTU6NYI1VUSV2THTIP18', 'riskScore': [0, 7, 4,

{'user_id': 'TCWJAVACQ2LK9SAZRH6R', 'riskScore': [8, 7, 1, 0, 8, 8], 'watch_list_symbols': ['VIDHIING', 'INDOSOLAR', 'INDOSOLAR', 'AYMSYNTEX', 'DHUNINV', 'LYKALABS', 'RPOWER', 'CONCOR', 'ESSARSHPNG', 'EROSMEDIA'], 'cluster_label': 1}

{'user_id': 'ARB47JR9R0ATC2MJK0DH', 'riskScore': [8, 7, 1, 0, 8, 8], 'watch_list_symbols': ['ADHUNIKIND', 'TATASTLBSL', 'SMARTLINK', 'GOACARBON', 'ARCHIDPLY', 'ESSARSHPNG', 'GOACARBON', 'GOACARBON', 'DIGISPICE', 'AYMSYNTEX'], 'cluster_label': 1}

{'user_id': 'WPU6HJG1EXY0663GLK68', 'riskScore': [5, 7, 0, 0, 8, 7], 'watch_list_symbols': ['RELIGARE', 'GEPIL', 'ESSARSHPNG', 'GOLDENTOBC', 'GAL', 'ARCHIDPLY', 'RELIGARE', 'SRPL', 'ADHUNIKIND', 'GAL'], 'cluster_label': 1}

{'user_id': 'WG9JCWKPCV2T7QVGZH17', 'riskScore': [5, 7, 0, 0, 8, 7], 'watch_list_symbols': ['AYMSYNTEX', 'INDOTECH', 'TATASTLBSL', 'ARCHIDPLY', 'PUNJLLOYD', 'SJVN', 'HDIL', 'INDOSOLAR', 'MUTHOOTCAP', 'AXISCADES'], 'cluster_label': 1}

-----------------------------------------------------------


{'user_id': 'V5HEMDUKFB9WNVY0CPI9', 'riskScore': [5, 3, 4, 8, 8, 1], 'watch_list_symbols': ['VIDHIING', 'VIDHIING', 'SMARTLINK', 'SJVN', 'SMARTLINK', 'ESSARSHPNG', 'RUPA', 'DHUNINV', 'RUPA', 'AYMSYNTEX'], 'cluster_label': 1}

{'user_id': 'I8PH6B5S7W07GMW5NHDF', 'riskScore': [5, 3, 4, 6, 8, 1], 'watch_list_symbols': ['MANAKALUCO', 'LYKALABS', 'JAGRAN', 'SUPPETRO', 'GOLDENTOBC', 'AYMSYNTEX', 'SMARTLINK', 'MUTHOOTCAP', 'ESABINDIA', 'MANUGRAPH'], 'cluster_label': 1}

{'user_id': '4EO5SIORJ97KSYYA94LF', 'riskScore': [5, 3, 4, 6, 8, 1], 'watch_list_symbols': ['SRPL', 'ARCHIDPLY', 'HDIL', 'SUPPETRO', 'SMARTLINK', 'ESSARSHPNG', 'TATASTLBSL', 'JHS', 'SUPERHOUSE', 'ESABINDIA'], 'cluster_label': 1}

{'user_id': 'GI7VRG0Y3JWC75MXACFN', 'riskScore': [5, 3, 4, 6, 8, 1], 'watch_list_symbols': ['BCG', 'MADRASFERT', 'BCG', 'RELIGARE', 'TATASTLBSL', 'TATASTLBSL', 'RELIGARE', 'ESSARSHPNG', 'HUBTOWN', 'ESSARSHPNG'], 'cluster_label': 1}

{'user_id': 'WVA4SWNCXEID33CN67MP', 'riskScore': [5, 3, 4, 8, 8, 3],

In [19]:
#k-number of high performing stocks which we want to take from neighbours watch list
def get_stock_recommendation(user_id,cluster_label,k,clusters):
    cluster = clusters["cluster_"+str(cluster_label)]
    _,NN_map = make_NN_map(cluster,6)
    nn_users = NN_map[user_id]
    
    common_watch_list={}
    for user in nn_users:
        watch_list_symbols = user["watch_list_symbols"][:k]
        for symbol in watch_list_symbols:
            common_watch_list[symbol]=1
    
    pChanges = []
    for symbol,val in common_watch_list.items():
        try:
            q = nse.get_quote(symbol)
            print(symbol +" : ")
            print(q)
            pChanges.append((float(q["pChange"]),symbol))
        except Exception as e:
            print(symbol +" : ")
            print(e)
    
    print(pChanges)
    pChanges_sorted = sorted(pChanges,reverse=True)
    return pChanges_sorted[:10]
        
    
    
    

In [21]:
get_stock_recommendation("KGZKNPPZUD4H8USR2649",1,4,clusters)

BALAXI : 
{'pricebandupper': 579.5, 'symbol': 'BALAXI', 'applicableMargin': 100.0, 'bcEndDate': None, 'totalSellQuantity': 346.0, 'adhocMargin': 73.0, 'companyName': 'BALAXI PHARMACEUTICALS LIMITED', 'marketType': 'N', 'exDate': None, 'bcStartDate': None, 'css_status_desc': 'Listed', 'dayHigh': 550.25, 'basePrice': 551.95, 'securityVar': 23.5, 'pricebandlower': 524.4, 'sellQuantity5': 13.0, 'sellQuantity4': 2.0, 'sellQuantity3': 133.0, 'cm_adj_high_dt': '14-OCT-20', 'sellQuantity2': 42.0, 'dayLow': 527.05, 'sellQuantity1': 1.0, 'quantityTraded': 2489.0, 'pChange': '-3.05', 'totalTradedValue': 14.2, 'deliveryToTradedQuantity': 64.28, 'totalBuyQuantity': 940.0, 'averagePrice': 537.28, 'indexVar': None, 'cm_ffm': 149.03, 'purpose': None, 'buyPrice2': 533.05, 'secDate': '19-Mar-2021 15:00:00', 'buyPrice1': 533.1, 'high52': 1080.2, 'previousClose': 551.95, 'ndEndDate': None, 'low52': 64.35, 'buyPrice4': 531.05, 'buyPrice3': 531.5, 'recordDate': None, 'deliveryQuantity': 1600.0, 'buyPrice5':

TATASTLBSL : 
{'pricebandupper': 54.35, 'symbol': 'TATASTLBSL', 'applicableMargin': 23.48, 'bcEndDate': '21-SEP-20', 'totalSellQuantity': 1130413.0, 'adhocMargin': None, 'companyName': 'Tata Steel Bsl Limited', 'marketType': 'N', 'exDate': '11-SEP-20', 'bcStartDate': '15-SEP-20', 'css_status_desc': 'Listed', 'dayHigh': 47.25, 'basePrice': 45.3, 'securityVar': 19.98, 'pricebandlower': 36.25, 'sellQuantity5': 66636.0, 'sellQuantity4': 12675.0, 'sellQuantity3': 132646.0, 'cm_adj_high_dt': '03-MAR-21', 'sellQuantity2': 13933.0, 'dayLow': 43.6, 'sellQuantity1': 35469.0, 'quantityTraded': 7287444.0, 'pChange': 3.53, 'totalTradedValue': 3661.84, 'deliveryToTradedQuantity': 30.0, 'totalBuyQuantity': 1195220.0, 'averagePrice': 45.33, 'indexVar': None, 'cm_ffm': 1287.85, 'purpose': 'ANNUAL GENERAL MEETING', 'buyPrice2': 46.85, 'secDate': '19-Mar-2021 15:00:00', 'buyPrice1': 46.9, 'high52': 50.8, 'previousClose': 45.3, 'ndEndDate': None, 'low52': 15.15, 'buyPrice4': 46.75, 'buyPrice3': 46.8, 'rec

MUTHOOTCAP : 
{'pricebandupper': 464.2, 'symbol': 'MUTHOOTCAP', 'applicableMargin': 23.49, 'bcEndDate': None, 'totalSellQuantity': 26059.0, 'adhocMargin': None, 'companyName': 'Muthoot Capital Services Limited', 'marketType': 'N', 'exDate': '12-JUN-17', 'bcStartDate': None, 'css_status_desc': 'Listed', 'dayHigh': 394.65, 'basePrice': 386.85, 'securityVar': 19.99, 'pricebandlower': 309.5, 'sellQuantity5': 53.0, 'sellQuantity4': 55.0, 'sellQuantity3': 1.0, 'cm_adj_high_dt': '31-AUG-20', 'sellQuantity2': 145.0, 'dayLow': 371.0, 'sellQuantity1': 27.0, 'quantityTraded': 19098.0, 'pChange': 2.02, 'totalTradedValue': 75.99, 'deliveryToTradedQuantity': 38.6, 'totalBuyQuantity': 40210.0, 'averagePrice': 383.49, 'indexVar': None, 'cm_ffm': 235.42, 'purpose': 'BONUS 1:10', 'buyPrice2': 392.7, 'secDate': '19-Mar-2021 15:00:00', 'buyPrice1': 392.75, 'high52': 528.0, 'previousClose': 386.85, 'ndEndDate': None, 'low52': 221.7, 'buyPrice4': 392.55, 'buyPrice3': 392.6, 'recordDate': '13-JUN-17', 'deliv

AYMSYNTEX : 
{'pricebandupper': 50.8, 'symbol': 'AYMSYNTEX', 'applicableMargin': 100.0, 'bcEndDate': '29-SEP-20', 'totalSellQuantity': 7755.0, 'adhocMargin': 70.55, 'companyName': 'AYM Syntex Limited', 'marketType': 'N', 'exDate': '24-SEP-20', 'bcStartDate': '26-SEP-20', 'css_status_desc': 'Listed', 'dayHigh': 47.45, 'basePrice': 48.4, 'securityVar': 25.95, 'pricebandlower': 46.0, 'sellQuantity5': 66.0, 'sellQuantity4': 250.0, 'sellQuantity3': 157.0, 'cm_adj_high_dt': '08-FEB-21', 'sellQuantity2': 1000.0, 'dayLow': 46.0, 'sellQuantity1': 2721.0, 'quantityTraded': 27896.0, 'pChange': '-4.96', 'totalTradedValue': 12.91, 'deliveryToTradedQuantity': 87.82, 'totalBuyQuantity': None, 'averagePrice': 46.24, 'indexVar': None, 'cm_ffm': 67.81, 'purpose': 'ANNUAL GENERAL MEETING', 'buyPrice2': None, 'secDate': '19-Mar-2021 15:00:00', 'buyPrice1': None, 'high52': 61.35, 'previousClose': 48.4, 'ndEndDate': None, 'low52': 13.15, 'buyPrice4': None, 'buyPrice3': None, 'recordDate': None, 'deliveryQua

[(3.67, 'AARTISURF'),
 (3.53, 'TATASTLBSL'),
 (3.34, 'SUPERHOUSE'),
 (2.96, 'SJVN'),
 (2.73, 'CONCOR'),
 (2.02, 'MUTHOOTCAP'),
 (1.68, 'ESABINDIA'),
 (1.36, 'MADRASFERT'),
 (0.64, 'PTL'),
 (-0.13, 'SUPPETRO')]